In [8]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pinecone,

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'pinecone,': Expected end or semicolon (after name and no valid version specifier)
    pinecone,
            ^


In [11]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\245642\anaconda3\envs\Medicalbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [13]:
PINECONE_API_KEY="pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2"
PINECONE_API_ENV="quickstart"

import os
from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2'



pc = pinecone.Pinecone(
    api_key= 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2',
)

if 'llama' not in pc.list_indexes().names():
   pc.create_index(
            name='llama',
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

In [14]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [15]:
#ectract data from pdf
def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [16]:
extracted_data = load_pdf("data/")

In [34]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2500, chunk_overlap = 200)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [35]:
text_chunks = text_split(extracted_data)
print("length of my chunks", len(text_chunks))

length of my chunks 9292


In [36]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [37]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [38]:
embeddings = download_hugging_face_embeddings()

In [39]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [40]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [41]:
pip install --upgrade langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Initializing the Pinecone
import os
from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2'



pc = pinecone.Pinecone(
    api_key= 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2',
)

index_name="mchat"

#Creating Embeddings for Each of The Text Chunks & storing

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])
